<a href="https://colab.research.google.com/github/rochactivo-UPC/dynLAB/blob/main/P4_Din_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4.  Sistemas de referencia no-inerciales


## Inicio: Librerías y datos necesarios

In [16]:
import pandas as pd
import numpy as np
from scipy.integrate import cumtrapz

# URL del archivo CSV en el repositorio de GitHub
url = 'https://raw.githubusercontent.com/rochactivo-UPC/dynLAB/main/P4/P4_Data.csv'

# Descargar el archivo CSV e importarlo en un DataFrame de Pandas
df = pd.read_csv(url, header=0, sep='\s+')

# Unidades en m/s2

In [17]:
df.head()

,t,Ax,Ay,Az,wx,wy,wz,ax,ay,az,R11,R12,R13,R21,R22,R23,R31,R32,R33
0,0.0000,-4.478,-6.935,5.672,0.0307,0.7855,0.0563,0.0000,0.0000,0.0000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0001,-4.784,-7.107,0.904,0.0292,0.7562,0.0669,-0.0156,-0.2936,0.1063,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0002,-3.494,-5.802,-4.093,0.0244,0.7424,0.0597,-0.0481,-0.1374,-0.0726,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0003,-2.259,-2.949,-7.271,0.0610,0.7720,0.0146,0.3658,0.2958,-0.4508,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0004,-2.684,0.420,-11.451,0.1431,0.8430,-0.0800,0.8212,0.7095,-0.9454,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


### Las funciones para estimar el HIC, r_0 y GHIC se muestran aquí:

In [37]:
import numpy as np
from scipy.integrate import simps
from numpy.linalg import norm

def calculate_ghic(a, alpha, omega, r0, time, delta_t=0.015, g=9.81):
    """
    Calculate the Generalized Head Injury Criterion (GHIC) for a given set of linear acceleration (a),
    angular velocity (omega), angular acceleration (alpha), initial position (r0), and time data.

    Parameters:
    - a: Numpy array of linear acceleration vectors.
    - alpha: Numpy array of angular acceleration vectors.
    - omega: Numpy array of angular velocity vectors.
    - r0: Numpy array of position vector.
    - time: Numpy array of time values.
    - delta_t: Time interval for GHIC calculation (default is 0.015s or 15ms).

    Returns:
    - The GHIC value.
    """
    # Number of time steps
    n = len(time)
    ghic_max = 0

    a = a/g
    omega = omega
    alpha = alpha


    # Iterate through each time step
    for start_index in range(n):
        # Find the end index such that the time difference is approximately delta_t
        end_index = start_index
        while end_index < n and (time[end_index] - time[start_index]) < delta_t:
            end_index += 1

        # If the time window is exactly delta_t, calculate the GHIC for this window
        if end_index < n and np.isclose(time[end_index] - time[start_index], delta_t, atol=1e-4):
            # Calculate the integral part of the formula for this window
            integrand = np.zeros(3)  # Initialize the integrand vector
            for i in range(start_index, end_index):
                integrand += (a[i] + np.cross(alpha[i], r0[i])/g + np.cross(omega[i], np.cross(omega[i], r0[i]))/g) * (time[i+1] - time[i])
                # integrand += (a[i]) * (time[i+1] - time[i])
            # Calculate the GHIC value for this window
            ghic_candidate = (norm(integrand) / delta_t) ** 2.5 * delta_t
            # Update the maximum GHIC value
            ghic_max = max(ghic_max, ghic_candidate)

    return ghic_max

# Example usage:
# a = np.array(...)     # Your linear acceleration data as an array of vectors
# alpha = np.array(...) # Your angular acceleration data as an array of vectors
# omega = np.array(...) # Your angular velocity data as an array of vectors
# r0 = np.array(...)    # Your initial position vector
# time = np.array(...)  # Your time data as a 1D array
# ghic_value = calculate_ghic(a, alpha, omega, r0, time)
# print("GHIC value:", ghic_value)


In [19]:
import numpy as np

def calculate_r0(omega, v):
    """
    Calculate the initial position vector r0 based on the angular velocity vectors omega and
    the linear velocity vectors v for each timestep.

    Parameters:
    - omega: Numpy array of shape (N, 3) representing the angular velocity vectors.
    - v: Numpy array of shape (N, 3) representing the linear velocity vectors.

    Returns:
    - r0: Numpy array of shape (N, 3) representing the initial position vectors.
    """
    # Calculate the magnitude squared of the angular velocity vector for each timestep
    omega_magnitude_squared = np.sum(omega**2, axis=1)

    # Replace zeros in omega_magnitude_squared to avoid division by zero
    omega_magnitude_squared[omega_magnitude_squared == 0] = np.nan

    # Calculate the cross product of omega and v for each timestep
    cross_product = np.cross(omega, v, axis=1)

    # Calculate r0 for each timestep using the cross product divided by the magnitude squared of omega
    r0 = cross_product / omega_magnitude_squared[:, np.newaxis]

    return r0

# Example usage:
# Assume that 'omega' and 'v' are already defined as numpy arrays of shape (N, 3)
# omega = np.array(...)  # Replace with your (N, 3) array of angular velocity vectors
# v = np.array(...)      # Replace with your (N, 3) array of linear velocity vectors
# r0 = calculate_r0(omega, v)

In [20]:
import numpy as np
from scipy.integrate import simps

def calculate_hic(acceleration, time, g=9.81):
    """
    Calculate the Head Injury Criterion (HIC) for a given set of acceleration data over time.

    Parameters:
    - acceleration: array of acceleration values
    - time: array of time values
    - g: gravity (m/s/s)

    Returns:
    - The HIC value.
    """
    acceleration = acceleration/g

    # Inicializar la variable HIC a un número muy pequeño
    hic = 0
    # El tiempo de integración debe ser menor o igual a 15 ms
    delta_t = 0.015
    # Calcular la integral de la aceleración para cada intervalo de tiempo posible
    for start in range(len(time)):
        for end in range(start + 1, len(time)):
            if time[end] - time[start] <= delta_t:
                # Integrar la aceleración en el intervalo de tiempo dado
                integral = simps(acceleration[start:end+1], time[start:end+1])
                # Calcular el valor de HIC para este intervalo y elevarlo a 2.5
                hic_candidate = (integral / (time[end] - time[start])) ** 2.5 * (time[end] - time[start])
                # Actualizar HIC si encontramos un valor más grande
                hic = max(hic, hic_candidate)

    return hic

## a) Cálculo de la matriz de velocidad angular $Ω_t$  a partir de los datos experimentales, integración numérica para obtener la matriz de rotación $R_t$
La matriz de rotación $R_t$ ser determina a partir de la matriz de velocidad angular en sistema inercial, definida como $\Omega_t = \dot R_t R_t $


## b)	Cálculo de la aceleración de la cabeza respecto al sistema de referencia inercial dado por el suelo.
Utilizando la matriz de rotación calculada en función del tiempo en el apartado anterior es posible convertir las aceleraciones del sistema no inercial a un sistema inercial
Para llevar del sistema no inercial (cabeza) al sistema inercial (suelo) se deberá rotar el vector de aceleración. La primer componente rotada del vector aceleración se estima como:

$$A_ix=R11*Ax+R12*Ay+R13*Az$$


In [11]:
df.head()

,t,Ax,Ay,Az,wx,wy,wz,ax,ay,az,...,R33,Aix,Aiy,Aiz,wix,wiy,wiz,aix,aiy,aiz
0,0.0000,-4.478,-6.935,5.672,0.0307,0.7855,0.0563,0.0000,0.0000,0.0000,...,1.0,-4.478,-6.935,5.672,0.0307,0.7855,0.0563,0.0000,0.0000,0.0000
1,0.0001,-4.784,-7.107,0.904,0.0292,0.7562,0.0669,-0.0156,-0.2936,0.1063,...,1.0,-4.784,-7.107,0.904,0.0292,0.7562,0.0669,-0.0156,-0.2936,0.1063
2,0.0002,-3.494,-5.802,-4.093,0.0244,0.7424,0.0597,-0.0481,-0.1374,-0.0726,...,1.0,-3.494,-5.802,-4.093,0.0244,0.7424,0.0597,-0.0481,-0.1374,-0.0726
3,0.0003,-2.259,-2.949,-7.271,0.0610,0.7720,0.0146,0.3658,0.2958,-0.4508,...,1.0,-2.259,-2.949,-7.271,0.0610,0.7720,0.0146,0.3658,0.2958,-0.4508
4,0.0004,-2.684,0.420,-11.451,0.1431,0.8430,-0.0800,0.8212,0.7095,-0.9454,...,1.0,-2.684,0.420,-11.451,0.1431,0.8430,-0.0800,0.8212,0.7095,-0.9454


In [28]:
df['Aix'] = df['R11']*df['Ax']+df['R12']*df['Ay']+df['R13']*df['Az']
df['Aiy'] = df['R21']*df['Ax']+df['R22']*df['Ay']+df['R23']*df['Az']
df['Aiz'] = df['R31']*df['Ax']+df['R32']*df['Ay']+df['R33']*df['Az']

Determinación del la velocidad angular en sistema inercial (en laboratorio)


In [29]:
df['wix'] = df['R11']*df['wx']+df['R12']*df['wy']+df['R13']*df['wz']
df['wiy'] = df['R21']*df['wx']+df['R22']*df['wy']+df['R23']*df['wz']
df['wiz'] = df['R31']*df['wx']+df['R32']*df['wy']+df['R33']*df['wz']

Determinación de la aceleración angular en sistema inercial (en laboratorio)

In [30]:
df['aix'] = df['R11']*df['ax']+df['R12']*df['ay']+df['R13']*df['az']
df['aiy'] = df['R21']*df['ax']+df['R22']*df['ay']+df['R23']*df['az']
df['aiz'] = df['R31']*df['ax']+df['R32']*df['ay']+df['R33']*df['az']

## c)	Determinación del HIC y predicción de efectos lesivos a partir del HIC (En casa)
Utilizando la ecuación conocida de las tres prácticas anteriores calcular el valor de HIC (En casa)

In [12]:
g = 9.81
Ai = np.sqrt((df['Aix'])**2 + (df['Aiy'])**2 + (df['Aiz'])**2)
time = df['t'].to_numpy()

In [21]:
HIC = calculate_hic(Ai, time)

In [22]:
print(HIC)

1011.2834897964397


## d)	Determinación del GHIC y predicción de los efectos lesivos a partir del GHIC (En casa)
para el cálculo de la distancia entre el eje de rotación y el centro de gravedad de la cabeza se usará $ \vec r_0 = ( \vec ω × \vec v )⁄‖\vec ω ‖^2 $, utilizar la función calculate_r0(omega, v) para ello. Obsérvese que la ecuación del HIC es un caso particular de la ecuación del GHIC eliminando los términos de información rotacional.

calculo de $ω$

In [26]:
omega = df[['wix','wiy','wiz']].to_numpy()

velocidad inicial es cero

In [31]:
df['vx'] = cumtrapz(df['Aix'], df['t'], initial=0)
df['vy'] = cumtrapz(df['Aiy'], df['t'], initial=0)
df['vz'] = cumtrapz(df['Aiy'], df['t'], initial=0)

In [33]:
v = df[['vx','vy','vz']].to_numpy()
r0 = calculate_r0(omega, v)

In [34]:
print(r0)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-8.38505649e-04 -1.81577873e-05  5.71229951e-04]
 [-1.65666104e-03 -3.50730711e-05  1.11324585e-03]
 ...
 [ 2.29353882e+00  1.39804598e+00 -1.31147434e+00]
 [ 2.30736515e+00  1.39063824e+00 -1.30351957e+00]
 [ 2.31944346e+00  1.38983070e+00 -1.30220677e+00]]


In [35]:
a = df[['Aix', 'Aiy', 'Aiz']].to_numpy()
alpha = df[['aix', 'aiy', 'aiz']].to_numpy()

In [38]:
ghic_value = calculate_ghic(a=a, alpha=alpha, omega=omega, r0=r0, time=time, delta_t=0.015, g=9.81)
print("GHIC value:", ghic_value)

GHIC value: 1264.022947485288


# Tareas a realizar, presentación del informe:

1) Representar las 3 componentes independientes de $Ω_t$ y las 6 componentes independientes de $R_t$ (dos gráficos uno para conjunto de componentes).

2) Presentar tres conjuntos de gráficas: para la aceleración $a ⃗_{ni} (t)$ (3 componentes) y $a ⃗_i (t)$ (3 componentes), compara los módulos de las aceleraciones obtenidas.

3) 	Usar la aceleración $a ⃗_i (t)$ obtenida para estimar una cota superior de HIC (Head Injury Criterion), tal como se hizo en la práctica 3.

4) Usar la aceleración $a ⃗_i (t)$ y la velocidad angular $ω ⃗(t)$ para computar una cota superior del GHIC (Generalized Head Injury Criterion). Comentar los resultados comparados de 3 y 4.

5) Explicar por qué para simular los esfuerzos en el cerebro necesitamos la aceleración respecto al suelo $a ⃗_i (t)$ y no podemos hacer servir directamente $a ⃗_{ni}$.

6) Propon una mejora a la práctica